# Imports and Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/ENGO_645/ENGO_645_Project_W2025/

/content/drive/MyDrive/ENGO_645/ENGO_645_Project_W2025


In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

In [6]:
Min_Occ = gpd.read_file('Data/Mineral_Occurences/mrds-fUS32.shp')
filtered_Min_Occ = Min_Occ[Min_Occ['score'].isin(['A', 'B', 'C', 'D'])]
filtered_Min_Occ = filtered_Min_Occ.drop(columns=['score', 'geometry', 'alteration', 'hrock_type'])
Min_Occ_df = pd.DataFrame(filtered_Min_Occ)
Min_Occ_df

,latitude,longitude,commod1,commod2,commod3
3,40.40325,-114.5008,Copper,None,None
4,41.01685,-114.9492,Tungsten,None,None
5,37.43914,-114.6383,Perlite,None,None
6,41.42319,-117.3842,Geothermal,None,None
8,37.90107,-114.4739,"Manganese, Silver, Lead",None,"Zinc, Vanadium, Copper, Arsenic"
...,...,...,...,...,...
15439,37.00716,-116.7933,"Gold, Silver","Lead, Copper",None
15440,38.54993,-117.0342,Gold,Silver,"Mercury, Arsenic, Antimony"
15441,38.06244,-117.2176,"Gold, Silver","Lead, Copper",Tungsten
15442,40.78319,-117.5176,"Silver, Gold","Lead, Zinc, Copper",None


In [7]:
commodity_columns = ["commod1", "commod2", "commod3"]
for col in commodity_columns:
    Min_Occ_df[col] = Min_Occ_df[col].fillna("").astype(str)

unique_minerals = set()
for col in commodity_columns:
    unique_minerals.update(Min_Occ_df[col].str.split(", ").explode().unique())

unique_minerals.discard("")

df_np = Min_Occ_df[commodity_columns].to_numpy()

binary_matrix = np.zeros((len(Min_Occ_df), len(unique_minerals) * len(commodity_columns)), dtype=np.uint8)

In [9]:
mineral_list = sorted(unique_minerals)
for row_idx, row in enumerate(df_np):
    for col_idx, minerals in enumerate(row):
        for mineral in minerals.split(", "):
            if mineral in unique_minerals:
                mineral_idx = mineral_list.index(mineral)
                binary_matrix[row_idx, mineral_idx + (col_idx * len(unique_minerals))] = 1

#Convert back to DataFrame with column names
binary_columns = [f"{mineral.lower()}_{i+1}" for i in range(len(commodity_columns)) for mineral in mineral_list]
encoded_df = pd.DataFrame(binary_matrix, columns=binary_columns)

#Concatenate with original data
Min_Occ_df = Min_Occ_df.reset_index(drop=True)  # Reset index of Min_Occ_df
encoded_df = encoded_df.reset_index(drop=True)
Min_Occ_df_encoded = pd.concat([Min_Occ_df.drop(columns=commodity_columns), encoded_df], axis=1)

Min_Occ_df_encoded

,latitude,longitude,aluminum_1,anthracite_1,antimony_1,arsenic_1,asbestos_1,barium-barite_1,bentonite_1,beryllium_1,...,travertine_3,tungsten_3,uranium_3,vanadium_3,vermiculite_3,volcanic materials_3,wollastonite_3,zeolites_3,zinc_3,zirconium_3
0,40.40325,-114.5008,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,41.01685,-114.9492,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37.43914,-114.6383,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41.42319,-117.3842,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37.90107,-114.4739,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14716,37.00716,-116.7933,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14717,38.54993,-117.0342,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14718,38.06244,-117.2176,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
14719,40.78319,-117.5176,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Model

In [10]:
GRID_SIZE = 0.05

#bounding box == lat/long of nevada
minx, miny, maxx, maxy = -120, 34, -114, 42

#create grid
x_coords = np.arange(minx, maxx, GRID_SIZE)
y_coords = np.arange(miny, maxy, GRID_SIZE)
grid_cells = [box(x, y, x + GRID_SIZE, y + GRID_SIZE) for x in x_coords for y in y_coords]

#create a GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame({"geometry": grid_cells}, crs="EPSG:4326")
#reset the index and rename index
grid_gdf = grid_gdf.reset_index().rename(columns={'index': 'grid_index'})

#convert to GeoDataFrame
Min_Occ_df_encoded["geometry"] = gpd.points_from_xy(Min_Occ_df_encoded["longitude"], Min_Occ_df_encoded["latitude"])
Min_Occ_gdf = gpd.GeoDataFrame(Min_Occ_df_encoded, geometry="geometry", crs="EPSG:4326")

#spatial join each mineral occurrence to a grid cell
grid_aggregated = gpd.sjoin(grid_gdf, Min_Occ_gdf, how="left", predicate="intersects")

#aggregate occurrences
grid_feature_matrix = grid_aggregated.groupby("grid_index").agg(
    {col: "sum" for col in Min_Occ_df_encoded.columns if col not in ["latitude", "longitude", "geometry", "grid_index"]}).fillna(0)

#the mean latitude and longitude for each grid cell
grid_centroids = grid_aggregated.groupby("grid_index")[["latitude", "longitude"]].mean()

#merge the mineral occurrences and centroids
grid_feature_matrix = pd.merge(grid_feature_matrix, grid_centroids, left_index=True, right_index=True, how='left')

#merge with the grid geometry
grid_feature_matrix = pd.merge(grid_feature_matrix, grid_gdf[["grid_index", "geometry"]], left_index=True, right_on='grid_index', how='left')

In [11]:
#reshape the dataset
grid_height = len(y_coords)
grid_width = len(x_coords)

#features and target labels
encoded_columns = [col for col in grid_feature_matrix.columns if col.endswith(('_1', '_2', '_3'))]
features = grid_feature_matrix[encoded_columns].values
labels = grid_feature_matrix["gold_1"].values

#reshape features
features = features.reshape(grid_height, grid_width, -1)
labels = labels.reshape(grid_height, grid_width, 1)

print("Feature shape:", features.shape)
print("Label shape:", labels.shape)

Feature shape: (160, 120, 330)
Label shape: (160, 120, 1)


In [12]:
features = features.reshape(grid_height * grid_width, features.shape[-1])
labels = labels.reshape(grid_height * grid_width, 1)

#Build CNN model
model = Sequential([
      Dense(128, activation="relu", input_shape=(features.shape[-1],)),
      Dropout(0.2),  #Add Dropout layer with a rate of 0.2
      Dense(64, activation="relu"),
      Dropout(0.2),  #Add another Dropout layer
      Dense(1, activation="linear")
  ])
#Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

#Train CNN
model.fit(features, labels, epochs=5, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
480/480 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9333 - loss: -3.0910 - val_accuracy: 0.9406 - val_loss: -1.3368
Epoch 2/5
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9423 - loss: -3.8148 - val_accuracy: 0.9753 - val_loss: -1.6883
Epoch 3/5
480/480 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9528 - loss: -3.0334 - val_accuracy: 0.9755 - val_loss: -1.7012
Epoch 4/5
480/480 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9520 - loss: -3.2846 - val_accuracy: 0.9753 - val_loss: -1.7006
Epoch 5/5
480/480 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9499 - loss: -3.0675 - val_accuracy: 0.9750 - val_loss: -1.6842


In [13]:
predictions = model.predict(features)
predictions = predictions.reshape(grid_height, grid_width)
threshold = 0.5

#Count the number of grid cells above the threshold
predicted_gold_sites_count = np.sum(predictions > threshold)

print("Number of predicted gold sites:", predicted_gold_sites_count)

600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Number of predicted gold sites: 1550


In [14]:
top_30_gold = grid_feature_matrix.sort_values(by=['gold_1'], ascending=False).head(20)

predictions = predictions.reshape(grid_height, grid_width)

#Find indices of top 30 predictions
top_30_pred_indices = np.unravel_index(np.argsort(predictions, axis=None)[-20:], predictions.shape)

#Get latitude and longitude for top 30 predictions
top_30_pred_lat = grid_feature_matrix['latitude'].values[top_30_pred_indices[0] * grid_width + top_30_pred_indices[1]]
top_30_pred_lon = grid_feature_matrix['longitude'].values[top_30_pred_indices[0] * grid_width + top_30_pred_indices[1]]

m = folium.Map(location=[38, -117], zoom_start=6)

#Create a dictionary to store locations
location_markers = {}

#Add markers for top 30 actual gold occurrences
for index, row in top_30_gold.iterrows():
    location = (row['latitude'], row['longitude'])

    #Check if location is in top 30 predictions
    if any(np.isclose(row['latitude'], top_30_pred_lat) & np.isclose(row['longitude'], top_30_pred_lon)):
        marker = folium.Marker(
            location=location,
            popup=f"Gold_1 Count: {row['gold_1']}, Predicted",
            icon=folium.Icon(color="purple", icon="info-sign")
        )
    else:
        marker = folium.Marker(
            location=location,
            popup=f"Gold_1 Count: {row['gold_1']}",
            icon=folium.Icon(color="red", icon="info-sign")
        )

    marker.add_to(m)
    location_markers[location] = marker

#Add markers for top 30 predicted gold sites
for lat, lon in zip(top_30_pred_lat, top_30_pred_lon):
    location = (lat, lon)
    if location not in location_markers:
        folium.Marker(
            location=[lat, lon],
            popup="Predicted Gold Site",
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

#Display the map
m